# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
#CQL Query file
from CQL_queries import *

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(f'Total number of rows:{len(full_data_rows_list)}')
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows:8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Explore event_datafile_new.csv

In [5]:
df = pd.read_csv("event_datafile_new.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [6]:
df.head(1).T

,0
artist,Barry Tuckwell/Academy of St Martin-in-the-Fie...
firstName,Mohammad
gender,M
itemInSession,0
lastName,Rodriguez
length,277.159
level,paid
location,"Sacramento--Roseville--Arden-Arcade, CA"
sessionId,961
song,Horn Concerto No. 4 in E flat K495: II. Romanc...


In [7]:
df.sample(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5114,Adam Lambert,Sara,F,20,Johnson,227.39546,paid,"Winston-Salem, NC",152,Whataya Want From Me,95
1972,Depeche Mode,Mohammad,M,22,Rodriguez,174.39302,paid,"Sacramento--Roseville--Arden-Arcade, CA",900,Jazz Thieves,88
1071,Blue October,Ayleen,F,0,Wise,325.90322,free,"Columbia, SC",212,Come In Closer,71
5709,Dr. Alban,Kinsley,F,9,Young,240.14322,paid,"Red Bluff, CA",977,It's My Life (Radio Edit),85
3065,Sondre Lerche,Aleena,F,19,Kirby,136.98567,paid,"Waterloo-Cedar Falls, IA",619,Face The Blood,44


# Datatype equivalent to Cassandra
Based on the info presented in event_datafile_new.csv these are the equivalent data types
and coresponding line column

- $0$ `artist`: TEXT
- $1$ `firstName`: TEXT
- $2$ `gender` : TEXT
- $3$ `itemInSession`: INT
- $4$ `lastName`: TEXT
- $5$ `length`: FLOAT
- $6$ `level`: TEXT
- $7$ `location`: TEXT
- $8$ `sessionId`: INT
- $9$ `song`: TEXT
- $10$`userId`: INT


## Sources

- [Cassandra datatypes](https://cassandra.apache.org/doc/latest/cql/types.html)
- [text v varchar](https://stackoverflow.com/questions/17530230/cassandra-text-vs-varchar)
- [float v double](https://stackoverflow.com/questions/58450530/in-cassandra-when-to-use-decimal-vs-float-double#:~:text=Cassandra%20has%20three%20types%20that,no%20upper%20bound%20on%20size)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

--------------------------------------------


# Create session table

![](images/session_table.PNG)
- drop table if exist
- create table 
## Columns
- session_id: INT
- item_in_session: INT
- user_id: INT
- artist_name : TEXT
- song_title: TEXT
- song_length : FLOAT
- PRIMARY KEYS( session_id, item_in_session)

In [11]:
# check/drop table 
session.execute(session_table_drop)

# create table
try:
    session.execute(session_table_create)
except Exception as e:
    print(e)

### Match dataframe columns with query

In [12]:
# get columns in question
df.iloc[:,[8,3,10,0,9,5]].sample(5)

,sessionId,itemInSession,userId,artist,song,length
2906,605,22,97,Pat Benatar,Don't Walk Away,290.37669
5752,1100,1,36,Hazell Dean,No Fool (For Love),367.98649
1884,781,0,44,Stray Cats,Ubangi Stomp,195.05587
1733,633,15,97,N.E.R.D.,Rock Star,259.23873
5676,1096,5,49,Tears For Fears,Head Over Heels,255.34649


# Insert data to tables from csv file
- call `session_table_insert` variable
- match column with line possision
    - session_id $-->$ `line[8]`
    - item_in_Session  $-->$ `line[3]`
    - user_id  $-->$ `line[10]`
    - artist_name  $-->$ `line[0]`
    - song_title  $-->$ `line[9]`
    - song_length  $-->$ `line[5]`

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(session_table_insert, (int(line[8]), int(line[3]), int(line[10]), str(line[0]), str(line[9]), float(line[5])))

# Answer Question 1
>1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

- call `session_table_query`
- append requested values
- print results

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:

try:
    rows = session.execute(session_table_query, (338, 4))
except Exception as e:
    print(e)
     
q1 = pd.DataFrame(list(rows))
q1

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Dataframe Solution

In [15]:
# pandas query result
q1_sol = df[(df.sessionId == 338) & (df.itemInSession == 4)]
q1_sol[['artist','song','length']]

,artist,song,length
444,Faithless,Music Matters (Mark Knight Dub),495.3073


# Create user table

![](images/user_table.PNG)
- drop table if exist
- create table 
### Columns
- user_id : INT
- session_id: INT
- item_in_session: INT
- user_first_name : TEXT
- user_last_name : TEXT
- artist_name : TEXT
- song_title: TEXT
- PRIMARY KEYS((user_id,session_id), item_in_session)

In [16]:
# check/drop table 
session.execute(user_table_drop)

# create table
try:
    session.execute(user_table_create)
except Exception as e:
    print(e)

## Match dataframe columns with query

In [17]:
# get columns in question
df.iloc[:,[10,8,3,1,4,0,9]].sample(5)

,userId,sessionId,itemInSession,firstName,lastName,artist,song
5252,88,471,1,Mohammad,Rodriguez,Europe,The Final Countdown
2969,97,605,42,Kate,Harrell,3 Doors Down,Kryptonite
4621,15,172,41,Lily,Koch,The Crests,16 Candles
107,36,957,40,Matthew,Jones,Charlie Wilson featuring Snoop Dogg,Let It Out
5082,53,52,2,Celeste,Williams,Mynt,Playa Haters


# Insert data to tables from csv file
- call `user_table_insert` variable
- match column with line possision
    - user_id : $-->$ `line[10]`
    - session_id: $-->$ `line[8]`
    - item_in_session: $-->$ `line[3]`
    - user_first_name : $-->$ `line[1]`
    - user_last_name : $-->$ `line[4]`
    - artist_name : $-->$ `line[0]`
    - song_title: $-->$ `line[9]`

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(user_table_insert, (int(line[10]), int(line[8]), int(line[3]), line[1],line[4],line[0], line[9]))

# Answer Question 2
>2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [19]:
try:
    rows = session.execute(user_table_query, (10, 182))
except Exception as e:
    print(e)
    
q2 = pd.DataFrame(list(rows))
q2

,artist_name,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# Dataframe solution

In [20]:
q2_sol = df[(df.userId == 10) & (df.sessionId == 182)]
q2_sol[['artist','song','firstName','lastName']]

,artist,song,firstName,lastName
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
4705,Three Drives,Greece 2000,Sylvie,Cruz
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


# Create Song table

![](images/song_table.PNG)
- drop table if exist
- create table 
### Columns
- song_title : TEXT
- user_id : INT
- user_first_name TEXT
- user_last_name : TEXT
- PRIMARY KEY : song_title, user_id

In [21]:
# check/drop table 
session.execute(song_table_drop)

# create table
try:
    session.execute(song_table_create)
except Exception as e:
    print(e)

## Match dataframe columns with query

In [22]:
df.iloc[:,[9,10,1,4]].sample(5)

,song,userId,firstName,lastName
2666,Heartbreak Warfare,80,Tegan,Levine
2033,Vibrator,49,Chloe,Cuevas
559,Festival (Of 7 Lights),15,Lily,Koch
4943,The Funeral (Album Version),88,Mohammad,Rodriguez
3621,Sehr kosmisch,97,Kate,Harrell


# Insert data to tables from csv file
- call `song_table_insert` variable
- match column with line possision
    - song_title : $-->$ `line[9]`
    - user_id : $-->$ `line[10]`
    - user_first_name : $-->$ `line[1]`
    - user_last_name : $-->$ `line[4]`

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(song_table_insert, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

# Answer Question 3
>3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
# some string manipulation had to be made...
try:
    rows = session.execute(song_table_query%("'All Hands Against His Own'"))
except Exception as e:
    print(e)
    
    
q3 = pd.DataFrame(list(rows))
q3

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


# Dataframe Solution

In [25]:
q3_sol =df[df.song.str.contains("All Hands Against His Own")]
q3_sol[['firstName','lastName']]

,firstName,lastName
2792,Tegan,Levine
5135,Sara,Johnson
6298,Jacqueline,Lynch


### Drop the tables before closing out the sessions

In [26]:
try:
    session.execute(session_table_drop)
    session.execute(user_table_drop)
    session.execute(song_table_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()